### (1) Install Python Packages (Run Once)

Run the section below to install needed Python packages

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

### (2) Edit Tool Configuration and Execute

In [ ]:
### CONFIGURATION
configuration = {}

configuration["model"]                            = "bipolar"     # model geometry: monopolar or bipolar
configuration["pixel_size"]                       = 100           # pixel size
configuration["geometry"]                         = "Flat_human"  # geometry settings: HC/Pillar/Flat devices in pdish/rat/human

# configure input paths (either absolute paths or relative paths within the repository)
configuration["user_files_path"]                  = None          # If set to None, defaults to inner user_files directory
configuration["video_sequence_name"]              = [f"input_sample_{x+1}" for x in range(8)] #examplary input sequence
configuration["pixel_label_input_file"]           = f'image_sequence/pixel_label_PS{configuration["pixel_size"]}-lg.pkl'

# define input files for mono-polar arrays
configuration["monopolar"] = {
    "r_matrix_input_file_diagonal": f'r_matrix/COMSOL_results/{configuration["geometry"]}_PS{configuration["pixel_size"]}_EP_self.csv',
    "r_matrix_input_file_non_diagonal": f'r_matrix/COMSOL_results/{configuration["geometry"]}_PS{configuration["pixel_size"]}_EP_Rmat.csv'
}

# define input files for bi-polar arrays
configuration["bipolar"] = {
    "r_matrix_input_file_active": f'r_matrix/COMSOL_results/{configuration["geometry"]}_PS{configuration["pixel_size"]}_UCD_active.csv',
    "r_matrix_input_file_return": f'r_matrix/COMSOL_results/{configuration["geometry"]}_PS{configuration["pixel_size"]}_UCD_return.csv',
    "r_matrix_input_file_return_neighbor": f'r_matrix/COMSOL_results/{configuration["geometry"]}_PS{configuration["pixel_size"]}_UCD_return_neighbor.csv',
    "r_matrix_input_file_px_pos": f'r_matrix/COMSOL_results/PS{configuration["pixel_size"]}-lg_pos.csv'
}

# configure output paths (either absolute paths or relative paths within the repository)
configuration["r_matrix_output_file"]             = f'r_matrix/R_{configuration["geometry"]}_PS{configuration["pixel_size"]}.pkl'
configuration["netlist_output_file"]              = "netlist.sp"
configuration["output_prefix"]                    = "run"       # this prefix will be added to all generated output directories


# R matrix parameters
configuration["r_matrix_conductivity"]            = 1           # conductivity scaling factor of the electrolyte

# Pixel layout Parameters
configuration["photosensitive_area_edge_to_edge"] = 92          # edge-to-edge size of the photosensitive area
configuration["active_electrode_radius"]          = 17          # radius of the active electrode in um
configuration["light_to_current_conversion_rate"] = 0.4         # light to current conversion rate in A/W
configuration["photosensitive_area"]              = 4075.72     # total photosensitive area in um^2. Assign "None" for auto calculation

# Circuit Parameters
configuration["return_to_active_area_ratio"]      = 2           # monopolar only: ratio between return area and total active area
configuration["additional_edges"]                 = 142         # bipolar only: edge segments of the return
configuration["sirof_capacitance"]                = 6           # SIROF capacitance in mF/cm^2
configuration["initial_Vactive"]                  = .4          # Initial bias of the active electrode in V
configuration["Ipho_scaling"]                     = [0.4*(1+2.3/x) for x in [0.7, 1.4, 2.1, 2.8, 4.2, 5.6, 7.7, 9.8]]
                                                                # artificial scalar of photocurrent, useful for parametric sweep (e.g. computing S-D curve)
configuration["number_of_diodes"]                 = 2           # number of photo diodes per pixel
configuration["Isat"]                             = 0.02        # diode saturation current in pA
configuration["ideality_factor"]                  = 1.14        # ideality factor n of the diode
configuration["shunt_resistance"]                 = 790.15E3    # shunt resistance in Ohm. Assign "None" if no shunt
configuration["temperature"]                      = 37
configuration["nominal_temperature"]              = 25
configuration["simulation_duration"]              = .72         # simulation duration in seconds


### EXECUTION
from RPSim import run_rpsim
run_stages = None # available stages: ["resistive_mesh", "current_sequence", "circuit", "simulation", "plot_results"]. None = run all
run_rpsim(configuration=configuration, run_stages=run_stages, find_similar_runs=True)